In [1]:
import os
import cv2
import torch
import numpy as np
import timm
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# Helper function to find the arousal file by prefix
def find_arousal_file(subject_folder):
    for file in os.listdir(subject_folder):
        if "Arousal" in file:  # Match prefix 'Arousal'
            return os.path.join(subject_folder, file)
    raise FileNotFoundError(f"No file with prefix 'Arousal' found in {subject_folder}")

# Function to process arousal CSV files and save below-threshold values
def process_arousal_csvs(input_dir, output_dir, threshold=-250):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    subjects = sorted(os.listdir(input_dir))
    for subject in subjects:
        subject_folder = os.path.join(input_dir, subject)
        if not os.path.isdir(subject_folder):
            continue

        try:
            arousal_file = find_arousal_file(subject_folder)
            arousal_data = pd.read_csv(arousal_file)

            # Filter rows with arousal value below the threshold
            filtered_data = arousal_data[arousal_data['arousal'] < threshold].copy()
            if filtered_data.empty:
                print(f"Skipping {subject}: No arousal value below {threshold}.")
                continue

            filtered_data['arousal'] = 1  # Set arousal to 1 as per requirement

            # Save the processed data to a new CSV file
            subject_output_folder = os.path.join(output_dir, subject)
            if not os.path.exists(subject_output_folder):
                os.makedirs(subject_output_folder)

            output_csv_path = os.path.join(subject_output_folder, f"{os.path.basename(arousal_file).replace('.csv', '_Arousal1.csv')}")
            filtered_data.to_csv(output_csv_path, index=False)

            print(f"Processed and saved: {output_csv_path}")
        except FileNotFoundError as e:
            print(e)
            continue

# Dataset class for balanced extraction
class BalancedAttentionDataset(Dataset):
    def __init__(self, data_dir, fixed_sequence_length=12, transform=None, frame_rate=10):
        self.data_dir = data_dir
        self.fixed_sequence_length = fixed_sequence_length
        self.transform = transform
        self.frame_rate = frame_rate
        self.subjects = sorted(os.listdir(data_dir))[:390]  # Use the first 390 subjects
        self.data = self.preprocess_data()

    def preprocess_data(self):
        processed_data = []
        for subject in self.subjects:
            subject_folder = os.path.join(self.data_dir, subject)
            video_file = next((f for f in os.listdir(subject_folder) if f.endswith('.avi')), None)
            if not video_file:
                print(f"No video file found in {subject_folder}")
                continue
            video_path = os.path.join(subject_folder, video_file)
            try:
                arousal_path = find_arousal_file(subject_folder)
                arousal_data = pd.read_csv(arousal_path)
                attentive_times = [row['time'] for _, row in arousal_data.iterrows() if row['arousal'] >= -250]
                inattentive_times = [row['time'] for _, row in arousal_data.iterrows() if row['arousal'] < -250]
                if not attentive_times or not inattentive_times:
                    print(f"Skipping {subject}: Only one class present.")
                    continue
                minority, majority = (inattentive_times, attentive_times) if len(inattentive_times) < len(attentive_times) else (attentive_times, inattentive_times)
                grouped_majority = self.group_consecutive(majority, len(minority))
                if len(grouped_majority) < len(minority):
                    grouped_majority = self.group_consecutive(majority, 1)
                sampled_majority = grouped_majority[:len(minority)]
                processed_data.append({
                    "video_path": video_path,
                    "arousal_path": arousal_path,
                    "attentive_times": sampled_majority,
                    "inattentive_times": minority
                })
            except FileNotFoundError as e:
                print(e)
                continue
        return processed_data

    def group_consecutive(self, times, group_size):
        grouped = []
        temp_group = [times[0]]
        for i in range(1, len(times)):
            if times[i] - times[i - 1] == 1:
                temp_group.append(times[i])
            else:
                grouped.append(temp_group)
                temp_group = [times[i]]
        grouped.append(temp_group)
        flat_groups = [group[:group_size] for group in grouped if len(group) >= group_size]
        return flat_groups

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entry = self.data[idx]
        video_path = entry['video_path']
        arousal_data = pd.read_csv(entry['arousal_path'])
        if np.random.rand() > 0.5:
            times = entry['inattentive_times']
            label = 0
        else:
            times = entry['attentive_times']
            label = 1
        frames = self.extract_frames(video_path, arousal_data, times)
        if self.transform:
            frames = [self.transform(frame) for frame in frames]
        return torch.stack(frames[:self.fixed_sequence_length], 0), torch.tensor(label)

    def extract_frames(self, video_path, arousal_data, times):
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS) or 30
        frames = []
        for time_group in times:
            if not isinstance(time_group, list):
                time_group = [time_group]
            for time in time_group:
                frame_idx = int(time * fps)
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
                ret, frame = cap.read()
                if not ret:
                    continue
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, (224, 224))
                frame = torch.from_numpy(frame).permute(2, 0, 1).float() / 255.0
                frames.append(frame)
        cap.release()
        while len(frames) < self.fixed_sequence_length:
            frames.append(torch.zeros((3, 224, 224)))
        return frames[:self.fixed_sequence_length]

# Define ViT-based Model for Attention Detection
class ViTAttentionDetector(nn.Module):
    def __init__(self):
        super(ViTAttentionDetector, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        self.vit.head = nn.Identity()
        self.classifier = nn.Linear(768, 1)

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)
        x = self.vit(x)
        x = x.view(B, T, -1)
        x = x.mean(dim=1)
        x = self.classifier(x)
        return x.view(-1)

# Instantiate and process dataset
input_dir = "/content/drive/MyDrive/SEWA(AOML)"
output_dir = "/content/drive/MyDrive/SEWA(AOML)1"
process_arousal_csvs(input_dir, output_dir)


Skipping C1_S001_P001_AD4: No arousal value below -250.
Skipping C1_S001_P002_AD4: No arousal value below -250.
Processed and saved: /content/drive/MyDrive/SEWA(AOML)1/C1_S002_P003_AD4/Round1_SessionId_111_VideoId_15_UserId_7a52e85b_cdd1_4a0d_8918_5cc4703c5b69_Arousal_Arousal1.csv
Skipping C1_S002_P004_AD4: No arousal value below -250.
Skipping C1_S003_P005_AD4: No arousal value below -250.
Skipping C1_S003_P006_AD4: No arousal value below -250.
Skipping C1_S004_P007_AD4: No arousal value below -250.
Skipping C1_S004_P008_AD4: No arousal value below -250.
Skipping C1_S005_P009_AD4: No arousal value below -250.
Skipping C1_S005_P010_AD4: No arousal value below -250.
Processed and saved: /content/drive/MyDrive/SEWA(AOML)1/C1_S006_P011_AD4/Round1_SessionId_130_VideoId_15_UserId_0e51ac1f_2fd5_456d_b8a2_d2b4dcfc0fd9_Arousal_Arousal1.csv
Skipping C1_S006_P012_AD4: No arousal value below -250.
Skipping C1_S007_P013_AD4: No arousal value below -250.
Skipping C1_S007_P014_AD4: No arousal value 

In [2]:
import os
import pandas as pd


def create_arousal2_files(input_folder, output_folder, total_rows_required=9712, max_rows_per_file=25):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    all_rows = []

    # Step 1: Collect eligible rows from all files
    for subject_folder in os.listdir(input_folder):
        subject_path = os.path.join(input_folder, subject_folder)

        # Ensure it's a directory
        if os.path.isdir(subject_path):
            for file_name in os.listdir(subject_path):
                if file_name.endswith("Arousal.csv"):  # Process only Arousal CSV files
                    file_path = os.path.join(subject_path, file_name)

                    try:
                        # Read the CSV file
                        df = pd.read_csv(file_path)

                        # Ensure the required column exists
                        if 'arousal' not in df.columns:
                            print(f"Skipping {file_path}: Missing 'arousal' column.")
                            continue

                        # Filter rows with arousal > -250
                        eligible_rows = df[df['arousal'] > -250]

                        # Limit to max rows per file
                        all_rows.extend(eligible_rows.head(max_rows_per_file).values.tolist())

                    except Exception as e:
                        print(f"Error reading {file_path}: {e}")

    # Step 2: Cycle back if needed to reach total rows required
    row_count = len(all_rows)
    while row_count < total_rows_required:
        for subject_folder in os.listdir(input_folder):
            subject_path = os.path.join(input_folder, subject_folder)
            if os.path.isdir(subject_path):
                for file_name in os.listdir(subject_path):
                    if file_name.endswith("Arousal.csv"):  # Process only Arousal CSV files
                        file_path = os.path.join(subject_path, file_name)

                        try:
                            # Read the CSV file
                            df = pd.read_csv(file_path)
                            if 'arousal' not in df.columns:
                                continue

                            # Filter rows with arousal > -250 and skip already used rows
                            unused_rows = df[df['arousal'] > -250]
                            unused_rows = unused_rows.iloc[max_rows_per_file:]
                            all_rows.extend(unused_rows.values.tolist())
                            row_count = len(all_rows)

                            if row_count >= total_rows_required:
                                break
                        except Exception as e:
                            continue
            if row_count >= total_rows_required:
                break

    # Step 3: Create new CSV files in the output folder
    processed_rows = 0
    for subject_folder in os.listdir(input_folder):
        subject_path = os.path.join(input_folder, subject_folder)
        if os.path.isdir(subject_path):
            output_subject_path = os.path.join(output_folder, subject_folder)
            os.makedirs(output_subject_path, exist_ok=True)

            for file_name in os.listdir(subject_path):
                if file_name.endswith("Arousal.csv"):  # Process only Arousal CSV files
                    file_path = os.path.join(subject_path, file_name)
                    output_file_name = f"{os.path.splitext(file_name)[0]}_Arousal2.csv"
                    output_file_path = os.path.join(output_subject_path, output_file_name)

                    # Select rows for this file
                    rows_to_write = all_rows[processed_rows:processed_rows + max_rows_per_file]
                    processed_rows += len(rows_to_write)

                    # If no rows are left to write, skip
                    if not rows_to_write:
                        continue

                    # Write rows to the new CSV file
                    df_out = pd.DataFrame(rows_to_write, columns=["time", "arousal"])
                    df_out["arousal"] = 0  # Set arousal value to 0
                    df_out.to_csv(output_file_path, index=False)

                    # Stop if all required rows are processed
                    if processed_rows >= total_rows_required:
                        break
            if processed_rows >= total_rows_required:
                break

    print(f"Processed rows: {processed_rows}")
    print(f"New CSV files created in: {output_folder}")


# Input and output folder paths
input_folder_path = "/content/drive/MyDrive/SEWA(AOML)"
output_folder_path = "/content/drive/MyDrive/SEWA(AOML)2"

# Run the function
create_arousal2_files(input_folder_path, output_folder_path)


Processed rows: 9725
New CSV files created in: /content/drive/MyDrive/SEWA(AOML)2


In [4]:
import os
import cv2
import torch
import numpy as np
import timm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import pandas as pd
import torch.cuda.amp as amp
from tqdm import tqdm

class BalancedAttentionDataset(Dataset):
    def __init__(self, video_dir, attentive_dir, notattentive_dir, fixed_sequence_length=12, transform=None, frame_rate=50, max_samples=1000):
        self.video_dir = video_dir
        self.attentive_dir = attentive_dir
        self.notattentive_dir = notattentive_dir
        self.fixed_sequence_length = fixed_sequence_length
        self.transform = transform
        self.frame_rate = frame_rate
        self.max_samples = max_samples

        self.subjects = sorted(os.listdir(video_dir))
        self.data = self.preprocess_data()

    def preprocess_data(self):
        processed_data = []
        attentive_count = 0
        non_attentive_count = 0

        # Use tqdm for preprocessing progress
        for subject in tqdm(self.subjects, desc="Processing Subjects"):
            # Early stopping if we've reached max samples
            if attentive_count + non_attentive_count >= self.max_samples:
                break

            video_path = os.path.join(self.video_dir, subject)
            if not os.path.isdir(video_path):
                continue

            # Match video file
            video_file = next((f for f in os.listdir(video_path) if f.endswith('.avi')), None)
            if not video_file:
                continue

            video_path = os.path.join(video_path, video_file)

            # Match attentive CSV
            attentive_path = os.path.join(self.attentive_dir, subject)
            attentive_file = next((f for f in os.listdir(attentive_path) if f.endswith('_Arousal2.csv')), None) if os.path.exists(attentive_path) else None

            # Match non-attentive CSV
            notattentive_path = os.path.join(self.notattentive_dir, subject)
            notattentive_file = next((f for f in os.listdir(notattentive_path) if f.endswith('_Arousal1.csv')), None) if os.path.exists(notattentive_path) else None

            # Process attentive times
            if attentive_file and attentive_count < self.max_samples // 2:
                attentive_csv = pd.read_csv(os.path.join(attentive_path, attentive_file))
                for time in attentive_csv['time']:
                    if attentive_count < self.max_samples // 2:
                        processed_data.append({
                            'video_path': video_path,
                            'start_time': time,
                            'label': 0  # Attentive
                        })
                        attentive_count += 1
                    else:
                        break

            # Process non-attentive times
            if notattentive_file and non_attentive_count < self.max_samples // 2:
                notattentive_csv = pd.read_csv(os.path.join(notattentive_path, notattentive_file))
                for time in notattentive_csv['time']:
                    if non_attentive_count < self.max_samples // 2:
                        processed_data.append({
                            'video_path': video_path,
                            'start_time': time,
                            'label': 1  # Not Attentive
                        })
                        non_attentive_count += 1
                    else:
                        break

        print(f"Processed Attentive Samples: {attentive_count}")
        print(f"Processed Non-Attentive Samples: {non_attentive_count}")
        return processed_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entry = self.data[idx]
        video_path = entry['video_path']
        start_time = entry['start_time']
        label = entry['label']

        frames = self.extract_frames(video_path, start_time)

        if self.transform:
            frames = [self.transform(frame) for frame in frames]

        return torch.stack(frames[:self.fixed_sequence_length], 0), torch.tensor(label, dtype=torch.float32)

    def extract_frames(self, video_path, start_time):
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS) or self.frame_rate
        frames = []

        # Extract frames for the specific second
        frame_idx = int(start_time * fps)
        for offset in range(self.fixed_sequence_length):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx + offset)
            ret, frame = cap.read()
            if not ret:
                frame = np.zeros((224, 224, 3), dtype=np.uint8)
            else:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, (224, 224))

            frame = torch.from_numpy(frame).permute(2, 0, 1).float() / 255.0
            frames.append(frame)

        cap.release()
        return frames

class ViTAttentionDetector(nn.Module):
    def __init__(self):
        super(ViTAttentionDetector, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        self.vit.head = nn.Identity()
        self.classifier = nn.Linear(768, 1)

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)
        x = self.vit(x)
        x = x.view(B, T, -1)
        x = x.mean(dim=1)
        x = self.classifier(x)
        return x.view(-1)

# Split dataset into train and validation
def split_dataset(dataset, train_ratio=0.8):
    train_size = int(train_ratio * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    return train_dataset, val_dataset

# Main Training Script
def main():
    # Directories
    video_dir = "/content/drive/MyDrive/SEWA(AOML)"
    attentive_dir = "/content/drive/MyDrive/SEWA(AOML)2"
    notattentive_dir = "/content/drive/MyDrive/SEWA(AOML)1"

    # Transforms
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Create Dataset
    dataset = BalancedAttentionDataset(
        video_dir,
        attentive_dir,
        notattentive_dir,
        transform=transform,
        max_samples=1000  # Limit samples to reduce processing time
    )

    # Split dataset
    train_dataset, val_dataset = split_dataset(dataset)

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)

    # Device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Model
    model = ViTAttentionDetector().to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

    # Training
    scaler = torch.cuda.amp.GradScaler() if device == 'cuda' else None

    best_val_loss = float('inf')
    for epoch in range(10):  # Reduced epochs
        # Training Phase
        model.train()
        train_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}", unit="batch")

        for frames, labels in progress_bar:
            frames, labels = frames.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.cuda.amp.autocast() if device == 'cuda' else torch.no_grad():
                outputs = model(frames)
                loss = criterion(outputs, labels)

            if scaler:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()

            train_loss += loss.item()

            # Update progress bar
            progress_bar.set_postfix({
                'Train Loss': f'{loss.item():.4f}',
            })

        # Validation Phase
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        val_progress_bar = tqdm(val_loader, desc="Validation", unit="batch")

        with torch.no_grad():
            for frames, labels in val_progress_bar:
                frames, labels = frames.to(device), labels.to(device)
                outputs = model(frames)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                preds = torch.sigmoid(outputs) >= 0.5
                correct += (preds == labels).float().sum()
                total += labels.size(0)

                # Update validation progress bar
                val_progress_bar.set_postfix({
                    'Val Loss': f'{loss.item():.4f}',
                })

        # Print epoch summary
        print(f"Epoch {epoch+1}: Train Loss = {train_loss/len(train_loader):.4f}, "
              f"Val Loss = {val_loss/len(val_loader):.4f}, "
              f"Val Accuracy = {correct/total:.4f}")

        # Save best model
        if val_loss/len(val_loader) < best_val_loss:
            best_val_loss = val_loss/len(val_loader)
            torch.save(model.state_dict(), 'best_attention_model.pth')
            print(f"Saved new best model with validation loss: {best_val_loss:.4f}")

if __name__ == "__main__":
    main()

Processing Subjects:  22%|██▏       | 88/397 [00:00<00:00, 344.70it/s]


Processed Attentive Samples: 500
Processed Non-Attentive Samples: 500


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

<ipython-input-4-5f0b8de404a8>:189: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if device == 'cuda' else None
Epoch 1:   0%|          | 0/50 [00:00<?, ?batch/s]<ipython-input-4-5f0b8de404a8>:203: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast() if device == 'cuda' else torch.no_grad():
Validation: 100%|██████████| 13/13 [02:18<00:00, 10.66s/batch, Val Loss=0.0093]


Epoch 1: Train Loss = 0.1639, Val Loss = 0.0387, Val Accuracy = 0.9900
Saved new best model with validation loss: 0.0387


Validation: 100%|██████████| 13/13 [02:15<00:00, 10.43s/batch, Val Loss=0.0001]


Epoch 2: Train Loss = 0.0351, Val Loss = 0.0355, Val Accuracy = 0.9950
Saved new best model with validation loss: 0.0355


In [9]:
import os
import cv2
import torch
import numpy as np
import timm
import torch.nn as nn
import torchvision.transforms as transforms

class ViTAttentionDetector(nn.Module):
    def __init__(self):
        super(ViTAttentionDetector, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        self.vit.head = nn.Identity()
        self.classifier = nn.Linear(768, 1)

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)
        x = self.vit(x)
        x = x.view(B, T, -1)
        x = x.mean(dim=1)
        x = self.classifier(x)
        return x.view(-1)

def test_video(video_path, model_path='best_attention_model.pth'):
    """
    Test video for attention detection

    Args:
    - video_path (str): Path to the video file to test
    - model_path (str): Path to the saved model weights

    Returns:
    - List of attention predictions
    """
    # Device setup
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Load model
    model = ViTAttentionDetector().to(device)

    # Use weights_only=True for security and to match how the model was likely saved
    try:
        # First, try loading state_dict directly
        model.load_state_dict(torch.load(model_path, map_location=device))
    except Exception:
        # If that fails, try loading with weights_only=True
        model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))

    model.eval()

    # Video capture
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 50
    total_seconds = total_frames / fps

    # Transforms
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Prediction storage
    attention_results = []

    # Process video second by second
    for second in range(int(total_seconds)):
        # Extract frames for this second
        frames = []
        for offset in range(12):  # 12 frames per second
            cap.set(cv2.CAP_PROP_POS_FRAMES, int((second + offset/12) * fps))
            ret, frame = cap.read()

            if not ret:
                frame = np.zeros((224, 224, 3), dtype=np.uint8)
            else:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, (224, 224))

            frame = transform(frame)
            frames.append(frame)

        # Convert frames to tensor
        frames_tensor = torch.stack(frames).unsqueeze(0).to(device)

        # Predict attention
        with torch.no_grad():
            output = model(frames_tensor)
            probability = torch.sigmoid(output).cpu().numpy()[0]

            # Classify as attentive or not attentive
            is_attentive = probability > 0.5

            attention_results.append({
                'second': second,
                'is_attentive': is_attentive,
                'probability': probability
            })

    cap.release()

    return attention_results

def print_attention_summary(attention_results):
    """
    Print summary of attention detection results

    Args:
    - attention_results (list): List of attention prediction dictionaries
    """
    print("\nAttention Detection Summary:")
    print("-" * 40)

    attentive_seconds = [
        result['second'] for result in attention_results if result['is_attentive']
    ]
    non_attentive_seconds = [
        result['second'] for result in attention_results if not result['is_attentive']
    ]

    print("Attentive Seconds:")
    print(", ".join(map(str, attentive_seconds)) if attentive_seconds else "None")

    print("\nNon-Attentive Seconds:")
    print(", ".join(map(str, non_attentive_seconds)) if non_attentive_seconds else "None")

    print(f"\nTotal Seconds Analyzed: {len(attention_results)}")
    print(f"Attentive Seconds: {len(attentive_seconds)}")
    print(f"Non-Attentive Seconds: {len(non_attentive_seconds)}")
    print(f"Attentiveness Percentage: {len(attentive_seconds)/len(attention_results)*100:.2f}%")

# Typical usage
def analyze_video(video_path):
    """
    Main function to analyze video attention

    Args:
    - video_path (str): Path to the video file

    Returns:
    - List of attention results
    """
    # Test the video
    attention_results = test_video(video_path)

    # Print summary
    print_attention_summary(attention_results)

    return attention_results

# If you want to use this as a standalone script, uncomment the following:
if __name__ == "__main__":
    video_path = "/content/drive/MyDrive/SEWA(AOML)/C4_S108_P215_AD4/Round1_SessionId_206_VideoId_4_UserId_0d2867ac-de24-49f9-85ce-15cb4c5c1d9c.avi"
    analyze_video(video_path)

<ipython-input-9-cfa629414cc8>:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))



Attention Detection Summary:
----------------------------------------
Attentive Seconds:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 51

Non-Attentive Seconds:
50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93

Total Seconds Analyzed: 94
Attentive Seconds: 51
Non-Attentive Seconds: 43
Attentiveness Percentage: 54.26%


In [13]:
import shutil
shutil.move('best_attention_model.pth', '/content/drive/MyDrive/v9_e2.pth')


'/content/drive/MyDrive/v9_e2.pth'